In [ ]:

# =============================================================================
# 간단하고 안전한 고급 전처리 파이프라인
# 기존 구조 + 새로운 특성 (에러 방지)
# =============================================================================

from datetime import datetime
import numpy as np 
import pandas as pd 
import glob 
import random 
import os 
import matplotlib.pyplot as plt 
import seaborn as sns 
import ast 
import warnings
warnings.filterwarnings('ignore') 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from scipy import stats
from scipy.signal import find_peaks
import torch
import tensorflow as tf

In [ ]:
# =============================================================================
# 기존 기본 설정 유지
# =============================================================================

# 현재 날짜 및 시간 가져오기
now = datetime.now()
timestamp = now.strftime("%m%d_%H%M")  # 예: 0517_1530

submission_folder = '/users/KTL/Desktop/dacon/submission/'
submission_file = f'submission_new_advanced_{timestamp}.csv'

# seed 고정 
SD = 42 
random.seed(SD) 
np.random.seed(SD) 
os.environ['PYTHONHASHSEED'] = str(SD)
tf.random.set_seed(SD)  # TensorFlow 시드 설정

# 파일 경로 설정 - VSCode 상대경로로 변경
base_folder =  '/users/KTL/Desktop/ETRI_lifelog_dataset'
folder = '/ch2025_data_items'
data_dir = base_folder + folder 

# Parquet 파일 전체 경로 리스트 
parquet_files = glob.glob(os.path.join(data_dir, 'ch2025_*.parquet')) 


In [ ]:
# =============================================================================
# 기존 데이터 로딩 방식 유지
# =============================================================================

# 파일 이름을 키로, DataFrame을 값으로 저장할 딕셔너리 
lifelog_data = {} 

# 파일별로 읽기 
for file_path in parquet_files: 
    name = os.path.basename(file_path).replace('.parquet', '').replace('ch2025_', '') 
    lifelog_data[name] = pd.read_parquet(file_path) 
    print(f"✅ Loaded: {name}, shape = {lifelog_data[name].shape}") 

# 딕셔너리에 있는 모든 항목을 독립적인 변수로 할당 
for key, df in lifelog_data.items(): 
    globals()[f"{key}_df"] = df 

# 메트릭스 파일 읽기
metrics_train = pd.read_csv(base_folder + '/ch2025_metrics_train.csv')
sample_submission = pd.read_csv(base_folder+'/ch2025_submission_sample.csv')

# ✅ 기준 쌍 (subject_id, lifelog_date) 
sample_submission['lifelog_date'] = pd.to_datetime(sample_submission['lifelog_date']) 
test_keys = set(zip(sample_submission['subject_id'], sample_submission['lifelog_date'].dt.date)) 

# ✅ DataFrame 별 timestamp 컬럼 수동 지정 
dataframes = { 
    'mACStatus': (mACStatus_df, 'timestamp'), 
    'mActivity': (mActivity_df, 'timestamp'), 
    'mAmbience': (mAmbience_df, 'timestamp'), 
    'mBle': (mBle_df, 'timestamp'), 
    'mGps': (mGps_df, 'timestamp'), 
    'mLight': (mLight_df, 'timestamp'), 
    'mScreenStatus': (mScreenStatus_df, 'timestamp'), 
    'mUsageStats': (mUsageStats_df, 'timestamp'), 
    'mWifi': (mWifi_df, 'timestamp'), 
    'wHr': (wHr_df, 'timestamp'), 
    'wLight': (wLight_df, 'timestamp'), 
    'wPedo': (wPedo_df, 'timestamp'), 
} 

# ✅ 기존 분리 함수 유지
def split_test_train(df, subject_col='subject_id', timestamp_col='timestamp'): 
    df[timestamp_col] = pd.to_datetime(df[timestamp_col], errors='coerce') 
    df = df.dropna(subset=[timestamp_col]) 
    df['date_only'] = df[timestamp_col].dt.date 
    df['key'] = list(zip(df[subject_col], df['date_only'])) 
    test_df = df[df['key'].isin(test_keys)].drop(columns=['date_only', 'key']) 
    train_df = df[~df['key'].isin(test_keys)].drop(columns=['date_only', 'key']) 
    return test_df, train_df 

# ✅ 기존 분할 결과 저장 
for name, (df, ts_col) in dataframes.items(): 
    print(f"⏳ {name} 분리 중...") 
    test_df, train_df = split_test_train(df.copy(), subject_col='subject_id', timestamp_col=ts_col) 
    globals()[f"{name}_test"] = test_df 
    globals()[f"{name}_train"] = train_df 
    print(f"✅ {name}_test → {test_df.shape}, {name}_train → {train_df.shape}") 

In [ ]:
# =============================================================================
# 🚀 안전한 고급 전처리 함수들 (기존 방식 개선)
# =============================================================================

def safe_process_mACStatus(df):
    """충전 패턴 안전한 고급 분석"""
    print("🔋 충전 패턴 안전 분석 중...")
    
    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['date'] = df['timestamp'].dt.date
    df['hour'] = df['timestamp'].dt.hour
    df = df.sort_values(['subject_id', 'timestamp'])
    
    results = []
    
    for (subj, date), group in df.groupby(['subject_id', 'date']):
        status = group['m_charging'].values
        hours = group['hour'].values
        
        # 기본 통계
        ratio_charging = status.mean()
        transitions = (status[1:] != status[:-1]).sum() if len(status) > 1 else 0
        
        # 연속 충전 길이
        charging_sessions = []
        current_session = 0
        for s in status:
            if s == 1:
                current_session += 1
            else:
                if current_session > 0:
                    charging_sessions.append(current_session)
                    current_session = 0
        if current_session > 0:
            charging_sessions.append(current_session)
        
        # 시간대별 충전 패턴
        charging_hours = hours[status == 1]
        
        # 새로운 고급 특성들
        features = {
            'subject_id': subj,
            'date': date,
            # 기본 특성
            'charging_ratio': ratio_charging,
            'charging_transitions': transitions,
            'avg_charging_duration': np.mean(charging_sessions) if charging_sessions else 0,
            'max_charging_duration': np.max(charging_sessions) if charging_sessions else 0,
            
            # 🆕 고급 특성
            'charging_sessions_count': len(charging_sessions),
            'charging_regularity': 1 / (np.std(charging_sessions) + 1) if charging_sessions else 0,
            'preferred_charging_hour': stats.mode(charging_hours)[0][0] if len(charging_hours) > 0 else 12,
            'charging_hour_diversity': len(np.unique(charging_hours)) if len(charging_hours) > 0 else 0,
            'night_charging_ratio': (charging_hours < 6).sum() / len(charging_hours) if len(charging_hours) > 0 else 0,
            'day_charging_ratio': ((charging_hours >= 6) & (charging_hours <= 18)).sum() / len(charging_hours) if len(charging_hours) > 0 else 0,
            'evening_charging_ratio': (charging_hours > 18).sum() / len(charging_hours) if len(charging_hours) > 0 else 0,
        }
        
        results.append(features)
    
    result_df = pd.DataFrame(results)
    print(f"   ✅ 충전 특성 생성: {result_df.shape}")
    return result_df

def safe_process_mActivity(df):
    """활동 패턴 안전한 고급 분석"""
    print("🏃 활동 패턴 안전 분석 중...")
    
    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['date'] = df['timestamp'].dt.date
    df['hour'] = df['timestamp'].dt.hour
    df = df.sort_values(['subject_id', 'timestamp'])
    
    results = []
    
    for (subj, date), group in df.groupby(['subject_id', 'date']):
        activities = group['m_activity'].values
        hours = group['hour'].values
        
        if len(activities) == 0:
            continue
        
        # 기본 통계
        activity_counts = pd.Series(activities).value_counts()
        dominant_activity = activity_counts.index[0] if len(activity_counts) > 0 else 0
        
        # 전이 분석
        transitions = (activities[1:] != activities[:-1]).sum() if len(activities) > 1 else 0
        
        # N-gram 패턴 (간단한 2-gram)
        bigrams = []
        if len(activities) >= 2:
            bigrams = [(activities[i], activities[i+1]) for i in range(len(activities)-1)]
            bigram_counts = pd.Series(bigrams).value_counts()
            top_bigram_freq = bigram_counts.iloc[0] / len(bigrams) if len(bigram_counts) > 0 else 0
        else:
            top_bigram_freq = 0
        
        # 시간대별 주요 활동
        morning_activities = activities[(hours >= 6) & (hours < 12)]
        afternoon_activities = activities[(hours >= 12) & (hours < 18)]
        evening_activities = activities[(hours >= 18) & (hours < 24)]
        
        features = {
            'subject_id': subj,
            'date': date,
            # 기본 특성
            'activity_diversity': len(activity_counts),
            'dominant_activity': dominant_activity,
            'activity_entropy': stats.entropy(activity_counts.values + 1e-10),
            
            # 🆕 고급 특성
            'activity_transitions': transitions,
            'activity_transition_rate': transitions / len(activities) if len(activities) > 0 else 0,
            'activity_stability': (activities == dominant_activity).mean(),
            'top_bigram_frequency': top_bigram_freq,
            
            # 시간대별 패턴
            'morning_main_activity': stats.mode(morning_activities)[0][0] if len(morning_activities) > 0 else -1,
            'afternoon_main_activity': stats.mode(afternoon_activities)[0][0] if len(afternoon_activities) > 0 else -1,
            'evening_main_activity': stats.mode(evening_activities)[0][0] if len(evening_activities) > 0 else -1,
            'morning_activity_diversity': len(np.unique(morning_activities)) if len(morning_activities) > 0 else 0,
            'afternoon_activity_diversity': len(np.unique(afternoon_activities)) if len(afternoon_activities) > 0 else 0,
            'evening_activity_diversity': len(np.unique(evening_activities)) if len(evening_activities) > 0 else 0,
        }
        
        results.append(features)
    
    result_df = pd.DataFrame(results)
    print(f"   ✅ 활동 특성 생성: {result_df.shape}")
    return result_df

def safe_process_mLight(df):
    """조도 센서 안전한 고급 분석"""
    print("💡 조도 패턴 안전 분석 중...")
    
    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['date'] = df['timestamp'].dt.date
    df['hour'] = df['timestamp'].dt.hour
    
    results = []
    
    for (subj, date), group in df.groupby(['subject_id', 'date']):
        light_values = group['m_light'].dropna().values
        hours = group['hour'].values
        
        if len(light_values) < 3:
            continue
        
        # 기본 통계
        light_mean = np.mean(light_values)
        light_std = np.std(light_values)
        light_range = np.max(light_values) - np.min(light_values)
        
        # 시간대별 분석
        morning_light = group[(group['hour'] >= 6) & (group['hour'] < 12)]['m_light'].mean()
        afternoon_light = group[(group['hour'] >= 12) & (group['hour'] < 18)]['m_light'].mean()
        evening_light = group[(group['hour'] >= 18) & (group['hour'] < 24)]['m_light'].mean()
        night_light = group[(group['hour'] >= 0) & (group['hour'] < 6)]['m_light'].mean()
        
        # 피크 분석 (안전한 방식)
        try:
            if len(light_values) > 5:
                peaks, _ = find_peaks(light_values, height=np.percentile(light_values, 70))
                valleys, _ = find_peaks(-light_values, height=-np.percentile(light_values, 30))
                peak_count = len(peaks)
                valley_count = len(valleys)
            else:
                peak_count = 0
                valley_count = 0
        except:
            peak_count = 0
            valley_count = 0
        
        features = {
            'subject_id': subj,
            'date': date,
            # 기본 특성
            'light_mean': light_mean,
            'light_std': light_std,
            'light_min': np.min(light_values),
            'light_max': np.max(light_values),
            'light_range': light_range,
            
            # 🆕 고급 특성
            'light_cv': light_std / (light_mean + 1e-10),
            'light_peaks_count': peak_count,
            'light_valleys_count': valley_count,
            'light_variability': np.var(light_values),
            
            # 시간대별 패턴
            'morning_light': morning_light if not np.isnan(morning_light) else 0,
            'afternoon_light': afternoon_light if not np.isnan(afternoon_light) else 0,
            'evening_light': evening_light if not np.isnan(evening_light) else 0,
            'night_light': night_light if not np.isnan(night_light) else 0,
            
            # 일주기 특성
            'circadian_amplitude': (afternoon_light - night_light) if not (np.isnan(afternoon_light) or np.isnan(night_light)) else 0,
            'dark_ratio': (light_values < np.percentile(light_values, 25)).mean(),
            'bright_ratio': (light_values > np.percentile(light_values, 75)).mean(),
        }
        
        results.append(features)
    
    result_df = pd.DataFrame(results)
    print(f"   ✅ 조도 특성 생성: {result_df.shape}")
    return result_df

def safe_process_wHr(df):
    """심박수 안전한 고급 분석 (HRV 포함)"""
    print("💓 심박수 안전 분석 중...")
    
    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['date'] = df['timestamp'].dt.date
    
    results = []
    
    for (subj, date), group in df.groupby(['subject_id', 'date']):
        all_hr = []
        
        # 심박수 데이터 파싱 (안전한 방식)
        for hr_data in group['heart_rate']:
            try:
                if isinstance(hr_data, str):
                    parsed = ast.literal_eval(hr_data)
                    hr_values = [int(hr) for hr in parsed if hr is not None and 40 <= int(hr) <= 200]
                    all_hr.extend(hr_values)
            except:
                continue
        
        if len(all_hr) < 5:
            continue
        
        hr_array = np.array(all_hr)
        
        # 기본 심박수 통계
        hr_mean = np.mean(hr_array)
        hr_std = np.std(hr_array)
        hr_range = np.max(hr_array) - np.min(hr_array)
        
        # HRV 지표 (안전한 방식)
        try:
            rr_intervals = 60000 / hr_array  # RR intervals in ms
            rr_diff = np.diff(rr_intervals)
            
            rmssd = np.sqrt(np.mean(rr_diff ** 2))  # RMSSD
            pnn50 = np.sum(np.abs(rr_diff) > 50) / len(rr_diff) * 100 if len(rr_diff) > 0 else 0  # pNN50
        except:
            rmssd = 0
            pnn50 = 0
        
        features = {
            'subject_id': subj,
            'date': date,
            # 기본 특성
            'hr_mean': hr_mean,
            'hr_std': hr_std,
            'hr_min': np.min(hr_array),
            'hr_max': np.max(hr_array),
            'hr_range': hr_range,
            
            # 🆕 HRV 특성
            'hr_rmssd': rmssd,
            'hr_pnn50': pnn50,
            'hr_cv': hr_std / hr_mean if hr_mean > 0 else 0,
            
            # 생리학적 지표
            'hr_stress_load': (hr_array > 100).mean(),
            'hr_recovery_ratio': (hr_array < 70).mean(),
            'hr_variability_index': hr_std / hr_mean if hr_mean > 0 else 0,
            
            # 분포 특성
            'hr_median': np.median(hr_array),
            'hr_q75': np.percentile(hr_array, 75),
            'hr_q25': np.percentile(hr_array, 25),
            'hr_iqr': np.percentile(hr_array, 75) - np.percentile(hr_array, 25),
        }
        
        results.append(features)
    
    result_df = pd.DataFrame(results)
    print(f"   ✅ 심박수 특성 생성: {result_df.shape}")
    return result_df

def safe_process_wPedo(df):
    """보행 패턴 안전한 고급 분석"""
    print("👟 보행 패턴 안전 분석 중...")
    
    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['date'] = df['timestamp'].dt.date
    df['hour'] = df['timestamp'].dt.hour
    
    results = []
    
    for (subj, date), group in df.groupby(['subject_id', 'date']):
        steps = group['step'].values
        speeds = group['speed'].dropna().values
        distances = group['distance'].values
        hours = group['hour'].values
        
        if len(steps) == 0:
            continue
        
        # 기본 통계
        total_steps = np.sum(steps)
        total_distance = np.sum(distances)
        avg_speed = np.mean(speeds) if len(speeds) > 0 else 0
        max_speed = np.max(speeds) if len(speeds) > 0 else 0
        
        # 활동 세션 분석
        active_threshold = 10
        active_periods = steps > active_threshold
        
        # 연속 활동 구간
        activity_sessions = []
        current_session = 0
        for is_active in active_periods:
            if is_active:
                current_session += 1
            else:
                if current_session > 0:
                    activity_sessions.append(current_session)
                    current_session = 0
        if current_session > 0:
            activity_sessions.append(current_session)
        
        # 시간대별 활동
        morning_steps = steps[(hours >= 6) & (hours < 12)].sum()
        afternoon_steps = steps[(hours >= 12) & (hours < 18)].sum()
        evening_steps = steps[(hours >= 18) & (hours < 24)].sum()
        
        features = {
            'subject_id': subj,
            'date': date,
            # 기본 특성
            'total_steps': total_steps,
            'total_distance': total_distance,
            'avg_speed': avg_speed,
            'max_speed': max_speed,
            'speed_variability': np.std(speeds) if len(speeds) > 0 else 0,
            
            # 🆕 고급 특성
            'active_sessions_count': len(activity_sessions),
            'avg_session_length': np.mean(activity_sessions) if activity_sessions else 0,
            'max_session_length': np.max(activity_sessions) if activity_sessions else 0,
            'activity_fragmentation': len(activity_sessions) / (np.sum(activity_sessions) + 1),
            'step_regularity': 1 / (np.std(steps) + 1),
            'step_consistency': (steps > 0).mean(),
            
            # 시간대별 패턴
            'morning_steps': morning_steps,
            'afternoon_steps': afternoon_steps,
            'evening_steps': evening_steps,
            'morning_steps_ratio': morning_steps / (total_steps + 1),
            'afternoon_steps_ratio': afternoon_steps / (total_steps + 1),
            'evening_steps_ratio': evening_steps / (total_steps + 1),
            
            # 강도 분석
            'high_intensity_ratio': (steps > np.percentile(steps, 80)).mean() if len(steps) > 0 else 0,
            'low_activity_ratio': (steps < np.percentile(steps, 20)).mean() if len(steps) > 0 else 0,
        }
        
        results.append(features)
    
    result_df = pd.DataFrame(results)
    print(f"   ✅ 보행 특성 생성: {result_df.shape}")
    return result_df

def safe_process_mScreenStatus(df):
    """스크린 사용 패턴 안전한 고급 분석"""
    print("📱 스크린 패턴 안전 분석 중...")
    
    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['date'] = df['timestamp'].dt.date
    df['hour'] = df['timestamp'].dt.hour
    
    results = []
    
    for (subj, date), group in df.groupby(['subject_id', 'date']):
        screen_states = group['m_screen_use'].values
        hours = group['hour'].values
        
        if len(screen_states) == 0:
            continue
        
        # 기본 통계
        usage_ratio = np.mean(screen_states)
        transitions = (screen_states[1:] != screen_states[:-1]).sum() if len(screen_states) > 1 else 0
        
        # 사용 세션 분석
        usage_sessions = []
        current_session = 0
        for state in screen_states:
            if state == 1:
                current_session += 1
            else:
                if current_session > 0:
                    usage_sessions.append(current_session)
                    current_session = 0
        if current_session > 0:
            usage_sessions.append(current_session)
        
        # 시간대별 사용 패턴
        usage_hours = hours[screen_states == 1]
        morning_usage = ((usage_hours >= 6) & (usage_hours < 12)).sum() / len(usage_hours) if len(usage_hours) > 0 else 0
        afternoon_usage = ((usage_hours >= 12) & (usage_hours < 18)).sum() / len(usage_hours) if len(usage_hours) > 0 else 0
        evening_usage = ((usage_hours >= 18) & (usage_hours < 24)).sum() / len(usage_hours) if len(usage_hours) > 0 else 0
        night_usage = ((usage_hours >= 0) & (usage_hours < 6)).sum() / len(usage_hours) if len(usage_hours) > 0 else 0
        
        features = {
            'subject_id': subj,
            'date': date,
            # 기본 특성
            'screen_usage_ratio': usage_ratio,
            'screen_transitions': transitions,
            
            # 🆕 고급 특성
            'screen_sessions_count': len(usage_sessions),
            'avg_session_duration': np.mean(usage_sessions) if usage_sessions else 0,
            'max_session_duration': np.max(usage_sessions) if usage_sessions else 0,
            'session_duration_std': np.std(usage_sessions) if usage_sessions else 0,
            'usage_intensity': np.sum(usage_sessions) / len(screen_states),
            'usage_fragmentation': len(usage_sessions) / (np.sum(usage_sessions) + 1),
            
            # 시간대별 패턴
            'morning_usage_rate': morning_usage,
            'afternoon_usage_rate': afternoon_usage,
            'evening_usage_rate': evening_usage,
            'night_usage_rate': night_usage,
            
            # 사용 패턴 특성
            'peak_usage_hour': hours[np.argmax(screen_states)] if len(screen_states) > 0 else 12,
            'usage_regularity': 1 / (transitions + 1),
            'screen_addiction_score': usage_ratio * len(usage_sessions),  # 사용률 × 세션 수
        }
        
        results.append(features)
    
    result_df = pd.DataFrame(results)
    print(f"   ✅ 스크린 특성 생성: {result_df.shape}")
    return result_df

In [ ]:
# =============================================================================
# 🔧 개인화 특성 추출기 (안전한 버전)
# =============================================================================

class SafePersonalizedExtractor:
    """안전한 개인화 특성 추출"""
    
    def __init__(self):
        self.user_baselines = {}
    
    def compute_baselines(self, df, value_cols):
        """사용자별 기준선 안전하게 계산"""
        for subject_id, group in df.groupby('subject_id'):
            baseline = {}
            for col in value_cols:
                if col in group.columns and group[col].notna().sum() > 2:  # 최소 3개 값 필요
                    try:
                        baseline[f'{col}_personal_mean'] = group[col].mean()
                        baseline[f'{col}_personal_std'] = group[col].std()
                        baseline[f'{col}_personal_median'] = group[col].median()
                    except:
                        continue
            
            if baseline:  # 빈 dict가 아닐 때만 저장
                self.user_baselines[subject_id] = baseline
    
    def extract_personal_deviations(self, df, value_cols):
        """개인 기준 편차 안전하게 추출"""
        df = df.copy()
        
        for col in value_cols:
            if col not in df.columns:
                continue
            
            # 개인별 Z-score (안전한 방식)
            personal_zscores = []
            above_medians = []
            
            for _, row in df.iterrows():
                subject_id = row['subject_id']
                if subject_id in self.user_baselines:
                    baseline = self.user_baselines[subject_id]
                    
                    # Z-score 계산
                    if f'{col}_personal_mean' in baseline and f'{col}_personal_std' in baseline:
                        mean_val = baseline[f'{col}_personal_mean']
                        std_val = baseline[f'{col}_personal_std']
                        if std_val > 1e-10:
                            zscore = (row[col] - mean_val) / std_val
                        else:
                            zscore = 0
                    else:
                        zscore = 0
                    
                    # 중위수 이상/이하
                    if f'{col}_personal_median' in baseline:
                        median_val = baseline[f'{col}_personal_median']
                        above_median = 1 if row[col] > median_val else 0
                    else:
                        above_median = 0
                else:
                    zscore = 0
                    above_median = 0
                
                personal_zscores.append(zscore)
                above_medians.append(above_median)
            
            df[f'{col}_personal_zscore'] = personal_zscores
            df[f'{col}_above_personal_median'] = above_medians
        
        return df

In [ ]:
# =============================================================================
# 🚀 메인 안전한 전처리 실행
# =============================================================================

def run_safe_advanced_preprocessing():
    """안전한 고급 전처리 실행"""
    print("🚀 안전한 고급 전처리 파이프라인 시작...")
    
    processed_dfs = []
    
    # 각 센서별 안전한 고급 전처리 실행
    sensor_processors = [
        ('mACStatus_df', safe_process_mACStatus),
        ('mActivity_df', safe_process_mActivity),
        ('mLight_df', safe_process_mLight),
        ('wHr_df', safe_process_wHr),
        ('wPedo_df', safe_process_wPedo),
        ('mScreenStatus_df', safe_process_mScreenStatus),
    ]
    
    for sensor_name, processor_func in sensor_processors:
        if sensor_name in globals() and len(globals()[sensor_name]) > 0:
            try:
                result_df = processor_func(globals()[sensor_name])
                if len(result_df) > 0 and 'date' in result_df.columns:
                    processed_dfs.append(result_df)
                    print(f"   ✅ {sensor_name} 처리 완료: {result_df.shape}")
                else:
                    print(f"   ⚠️ {sensor_name} 결과가 비어있음")
            except Exception as e:
                print(f"   ❌ {sensor_name} 처리 실패: {e}")
        else:
            print(f"   ⚠️ {sensor_name} 데이터 없음")
    
    # 특성 병합
    print("🔗 모든 센서 특성 안전하게 병합 중...")
    if processed_dfs:
        merged_df = processed_dfs[0].copy()
        print(f"   📊 첫 번째 DF: {merged_df.shape}")
        
        for i, df in enumerate(processed_dfs[1:], 2):
            print(f"   📊 {i}번째 DF 병합: {df.shape}")
            merged_df = pd.merge(merged_df, df, on=['subject_id', 'date'], how='outer')
            print(f"      → 병합 후: {merged_df.shape}")
    else:
        print("   ❌ 처리된 특성이 없어 빈 DataFrame 생성")
        merged_df = pd.DataFrame(columns=['subject_id', 'date'])
        return pd.DataFrame(), pd.DataFrame(), merged_df
    
    # 개인화 특성 추가
    print("👤 개인화 특성 안전하게 추출 중...")
    if len(merged_df) > 5:  # 최소 5개 행 필요
        try:
            personal_extractor = SafePersonalizedExtractor()
            numeric_cols = merged_df.select_dtypes(include=[np.number]).columns.tolist()
            if 'subject_id' in numeric_cols:
                numeric_cols.remove('subject_id')
            
            if len(numeric_cols) > 0:
                personal_extractor.compute_baselines(merged_df, numeric_cols[:10])  # 최대 10개 컬럼만
                merged_df = personal_extractor.extract_personal_deviations(merged_df, numeric_cols[:10])
                personal_features_count = len([col for col in merged_df.columns if 'personal' in col])
                print(f"   ✅ 개인화 특성 {personal_features_count}개 추가")
            else:
                print("   ⚠️ 수치형 컬럼 없어 개인화 특성 생략")
        except Exception as e:
            print(f"   ❌ 개인화 특성 추출 실패: {e}")
    
    # Train/Test 분할
    print("✂️ Train/Test 안전하게 분할 중...")
    try:
        # date 컬럼 확인 및 변환
        if 'date' not in merged_df.columns:
            print("   ❌ 'date' 컬럼이 없습니다.")
            return pd.DataFrame(), pd.DataFrame(), merged_df
        
        merged_df['date'] = pd.to_datetime(merged_df['date']).dt.date
        metrics_train['lifelog_date'] = pd.to_datetime(metrics_train['lifelog_date']).dt.date
        
        # Train 데이터
        train_df = pd.merge(
            metrics_train.rename(columns={'lifelog_date': 'date'}),
            merged_df,
            on=['subject_id', 'date'],
            how='inner'
        )
        
        # Test 데이터
        sample_submission['lifelog_date'] = pd.to_datetime(sample_submission['lifelog_date']).dt.date
        test_df = pd.merge(
            sample_submission[['subject_id', 'lifelog_date']].rename(columns={'lifelog_date': 'date'}),
            merged_df,
            on=['subject_id', 'date'],
            how='left'
        )
        
        print(f"✅ 안전한 고급 전처리 완료!")
        print(f"   - Train 데이터: {train_df.shape}")
        print(f"   - Test 데이터: {test_df.shape}")
        print(f"   - 추출된 특성 수: {len([col for col in merged_df.columns if col not in ['subject_id', 'date']])}")
        
        return train_df, test_df, merged_df
        
    except Exception as e:
        print(f"   ❌ Train/Test 분할 실패: {e}")
        return pd.DataFrame(), pd.DataFrame(), merged_df


In [ ]:
# =============================================================================
# 실행
# =============================================================================

if __name__ == "__main__":
    train_df_advanced, test_df_advanced, features_df_advanced = run_safe_advanced_preprocessing()
    
    print("\n🎉 안전한 고급 전처리 완료!")
    print(f"📊 결과 요약:")
    print(f"   - Train: {train_df_advanced.shape}")
    print(f"   - Test: {test_df_advanced.shape}")
    print(f"   - Features: {features_df_advanced.shape}")
    
    if len(features_df_advanced) > 0:
        feature_examples = [col for col in features_df_advanced.columns 
                           if col not in ['subject_id', 'date']][:15]
        
        print(f"\n🔬 생성된 고급 특성 예시 (상위 15개):")
        for i, feature in enumerate(feature_examples, 1):
            print(f"   {i:2d}. {feature}")
        
        # 특성 카테고리별 개수
        personal_features = len([col for col in features_df_advanced.columns if 'personal' in col])
        session_features = len([col for col in features_df_advanced.columns if 'session' in col])
        ratio_features = len([col for col in features_df_advanced.columns if 'ratio' in col])
        advanced_features = len([col for col in features_df_advanced.columns if any(keyword in col for keyword in ['rmssd', 'pnn50', 'fragmentation', 'regularity'])])
        
        print(f"\n📈 특성 카테고리별 통계:")
        print(f"   - 개인화 특성: {personal_features}개")
        print(f"   - 세션 기반: {session_features}개") 
        print(f"   - 비율 특성: {ratio_features}개")
        print(f"   - 고급 생체신호: {advanced_features}개")
        print(f"   - 총 특성 수: {len(feature_examples)}개")
    
    else:
        print("   ⚠️ 특성이 생성되지 않았습니다.")

In [ ]:
# =============================================================================
# 고급 전처리 데이터 + 기존 LightGBM 모델링 파이프라인
# =============================================================================

from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import math

print("🎯 고급 전처리 데이터로 기존 LightGBM 모델링 시작...")

In [ ]:
# =============================================================================
# 1. 데이터 준비
# =============================================================================

# ✅ 타겟 리스트 
targets_binary = ['Q1', 'Q2', 'Q3', 'S2', 'S3'] 
target_multiclass = 'S1' 

print(f"📊 데이터 현황:")
print(f"   - Train 데이터: {train_df_advanced.shape}")
print(f"   - Test 데이터: {test_df_advanced.shape}")

# ✅ feature 준비 
feature_cols = [col for col in train_df_advanced.columns 
                if col not in ['subject_id', 'sleep_date', 'date', 'Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']]

X_advanced = train_df_advanced[feature_cols].copy()
X_advanced.fillna(0, inplace=True)  # 결측값 처리 

test_X_advanced = test_df_advanced[feature_cols].copy()
test_X_advanced.fillna(0, inplace=True) 

print(f"📈 특성 정보:")
print(f"   - 사용 특성 수: {len(feature_cols)}")
print(f"   - Train 샘플: {X_advanced.shape[0]}")
print(f"   - Test 샘플: {test_X_advanced.shape[0]}")

# 컬럼 이름에서 특수 문자 제거/변환 
def sanitize_column_names(df): 
    df.columns = ( 
        df.columns 
        .str.replace(r"[^\w]", "_", regex=True)  # 특수문자 → _ 
        .str.replace(r"__+", "_", regex=True)  # 연속된 _ 제거 
        .str.strip("_")  # 앞뒤 _ 제거 
    ) 
    return df 

# 모든 입력에 적용 
X_advanced = sanitize_column_names(X_advanced) 
test_X_advanced = sanitize_column_names(test_X_advanced)

print(f"✅ 컬럼명 정규화 완료")


In [ ]:
# =============================================================================
# 2. LightGBM 기본 설정
# =============================================================================

# GridSearch를 위한 교차 검증 설정
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# LightGBM 기본 파라미터 설정
lgbm_params = {
    'n_estimators': 1000,
    'learning_rate': 0.03,
    'random_state': 42,
    'n_jobs': -1,
    'verbosity': -1
}

# 이진 분류 모델 성능 향상을 위한 그리드 서치 파라미터
binary_param_grid = {
    'learning_rate': [0.01, 0.03, 0.1],
    'n_estimators': [1000, 1500],
    'num_leaves': [50, 100],
    'max_depth': [-1, 5],
    'min_child_samples': [10, 30],
}

# 다중 분류 모델 성능 향상을 위한 그리드 서치 파라미터
multi_param_grid = {
    'learning_rate': [0.01, 0.03, 0.1],
    'n_estimators': [1000, 1500],
    'num_leaves': [50, 100],
    'max_depth': [-1, 5],
    'min_child_samples': [10, 30],
}

print(f"⚙️ LightGBM 파라미터 설정 완료")

In [ ]:
# =============================================================================
# 3. 기본 모델 학습 (Feature Importance 추출용)
# =============================================================================

print(f"\n{'='*60}")
print(f"🔥 1단계: 기본 모델 학습 (Feature Importance 추출)")
print(f"{'='*60}")

# 이진 분류 기본 모델
binary_models_basic = {}
basic_importance_dict = {}

for col in targets_binary:
    print(f"   🎯 {col} 기본 모델 학습 중...")
    y = train_df_advanced[col]
    
    # 기본 LightGBM 모델
    model = LGBMClassifier(objective='binary', **lgbm_params)
    model.fit(X_advanced, y)
    
    # Feature Importance 저장
    importance_df = pd.DataFrame({
        'feature': X_advanced.columns,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False).head(20)  # 20개
    
    binary_models_basic[col] = model
    basic_importance_dict[col] = importance_df
    
    print(f"      ✅ 완료 - 최고 중요도: {importance_df.iloc[0]['importance']:.4f} ({importance_df.iloc[0]['feature']})")

# S1 다중분류 기본 모델
print(f"   🎯 S1 다중분류 기본 모델 학습 중...")
y_multi = train_df_advanced['S1']
model_s1_basic = LGBMClassifier(objective='multiclass', num_class=len(y_multi.unique()), **lgbm_params)
model_s1_basic.fit(X_advanced, y_multi)

# S1 Feature Importance 저장
s1_importance = pd.DataFrame({
    'feature': X_advanced.columns,
    'importance': model_s1_basic.feature_importances_
}).sort_values('importance', ascending=False).head(20)

basic_importance_dict['S1'] = s1_importance
print(f"      ✅ 완료 - 최고 중요도: {s1_importance.iloc[0]['importance']:.4f} ({s1_importance.iloc[0]['feature']})")


In [ ]:
# =============================================================================
# 4. Feature Importance 시각화
# =============================================================================

print(f"\n📊 Feature Importance 시각화...")

# 이진분류 모델들의 Feature Importance 시각화
n_models = len(binary_models_basic)
n_cols = 2
n_rows = math.ceil(n_models / n_cols)

fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 8 * n_rows))
axes = axes.flatten() if n_models > 1 else [axes]

for idx, (col_name, importance_df) in enumerate(basic_importance_dict.items()):
    if col_name == 'S1':  # S1은 따로 처리
        continue
        
    sns.barplot(
        x='importance', y='feature', 
        data=importance_df.head(10),  # 상위 10개만 표시
        ax=axes[idx], 
        palette='viridis'
    )
    axes[idx].set_title(f'{col_name} - Top 10 Feature Importance', fontsize=14, fontweight='bold')
    axes[idx].set_xlabel('Importance Score', fontsize=12)
    axes[idx].set_ylabel('Features', fontsize=12)
    axes[idx].tick_params(axis='y', labelsize=10)
    axes[idx].tick_params(axis='x', labelsize=10)

# 남은 빈 subplot 제거
for j in range(len(binary_models_basic), len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

# S1 다중분류 Feature Importance 시각화
plt.figure(figsize=(12, 10))
sns.barplot(x='importance', y='feature', data=s1_importance.head(15), palette='viridis')
plt.title('S1 다중분류 모델 - Top 15 Feature Importance', fontsize=16, fontweight='bold')
plt.xlabel('Importance Score', fontsize=12)
plt.ylabel('Features', fontsize=12)
plt.tight_layout()
plt.show()

print("📊 Feature Importance 시각화 완료!")

In [ ]:
# =============================================================================
# 5. 개별 모델 학습 (각 타겟별 상위 특성 사용 + SMOTE + GridSearch)
# =============================================================================

print(f"\n{'='*60}")
print(f"🚀 2단계: 개별 모델 최적화 학습 (SMOTE + GridSearch)")
print(f"{'='*60}")

# 각 모델별 이진분류 결과를 저장할 딕셔너리
binary_models_optimized = {}
binary_preds_optimized = {}

# 이진분류 모델들 최적화
for model_name in targets_binary:
    target_col = model_name
    
    # 해당 모델의 상위 20개 중요 피처 선택
    top_features = basic_importance_dict[model_name]['feature'].tolist()
    
    # 중요 피처들로 데이터 필터링
    X_model_selected = X_advanced[top_features]
    test_X_model_selected = test_X_advanced[top_features]
    
    y = train_df_advanced[target_col]
    
    print(f"\n🎯 {model_name} 모델 최적화")
    print(f"   - 사용 특성 수: {X_model_selected.shape[1]}개")
    print(f"   - 훈련 샘플 수: {X_model_selected.shape[0]}개")
    print(f"   - 클래스 분포: {dict(y.value_counts().sort_index())}")
    
    # k_neighbors 자동 조정
    min_class_count = min(y.value_counts())
    if min_class_count <= 3:
        k_neighbors = max(1, min_class_count - 1)
        print(f"   ⚠️  소수 클래스 샘플이 적어 k_neighbors를 {k_neighbors}로 조정")
    else:
        k_neighbors = 3
    
    # SMOTE + LightGBM Pipeline
    pipeline = ImbPipeline([
        ('smote', SMOTE(random_state=42, k_neighbors=k_neighbors)),
        ('classifier', LGBMClassifier(objective='binary', **lgbm_params))
    ])
    
    # 파라미터 그리드 (classifier__ 접두사 추가)
    binary_param_grid_mod = {}
    for key, value in binary_param_grid.items():
        binary_param_grid_mod[f'classifier__{key}'] = value
    
    # 그리드 서치로 최적 파라미터 찾기
    grid_search = GridSearchCV(
        estimator=pipeline,
        param_grid=binary_param_grid_mod,
        scoring='f1_macro',
        cv=cv,
        verbose=0,
        n_jobs=-1
    )
    
    print(f"   ⏳ 모델 최적화 중... (SMOTE k_neighbors={k_neighbors})")
    grid_search.fit(X_model_selected, y)
    
    print(f"   ✅ 완료!")
    print(f"      - 최적 파라미터: {grid_search.best_params_}")
    print(f"      - 최고 F1 점수: {grid_search.best_score_:.4f}")
    
    # 최적 모델과 예측 결과 저장
    best_model = grid_search.best_estimator_
    predictions = best_model.predict(test_X_model_selected)
    
    binary_models_optimized[model_name] = best_model
    binary_preds_optimized[model_name] = predictions
    
    print(f"      - 예측 완료: {len(predictions)}개 샘플")
    print(f"      - 예측 클래스 분포: {dict(pd.Series(predictions).value_counts().sort_index())}")


In [ ]:
# =============================================================================
# 6. S1 다중분류 모델 최적화
# =============================================================================

print(f"\n🎯 S1 다중분류 모델 최적화")

target_col = 'S1'
y_multiclass = train_df_advanced[target_col]

# S1 상위 20개 중요 피처 선택
top_features_s1 = basic_importance_dict[target_col]['feature'].tolist()
X_s1_selected = X_advanced[top_features_s1]
test_X_s1_selected = test_X_advanced[top_features_s1]

print(f"   - 사용 특성 수: {X_s1_selected.shape[1]}개")
print(f"   - 훈련 샘플 수: {X_s1_selected.shape[0]}개")
print(f"   - 클래스 수: {len(y_multiclass.unique())}개")
print(f"   - 클래스 분포: {dict(y_multiclass.value_counts().sort_index())}")

# SMOTE 적용
smote = SMOTE(random_state=42, k_neighbors=3)
X_s1_smote, y_multiclass_smote = smote.fit_resample(X_s1_selected, y_multiclass)

print(f"   - SMOTE 적용 후 샘플 수: {X_s1_smote.shape[0]}개")
print(f"   - SMOTE 적용 후 클래스 분포: {dict(pd.Series(y_multiclass_smote).value_counts().sort_index())}")

# 다중분류용 모델 설정
multiclass_model = LGBMClassifier(
    objective='multiclass',
    metric='multi_logloss',
    **lgbm_params
)

# 그리드 서치로 최적 파라미터 찾기
grid_search_multiclass = GridSearchCV(
    estimator=multiclass_model,
    param_grid=multi_param_grid,
    scoring='f1_macro',
    cv=cv,
    verbose=0,
    n_jobs=-1
)

print(f"   ⏳ S1 다중분류 모델 최적화 중...")
grid_search_multiclass.fit(X_s1_smote, y_multiclass_smote)

print(f"   ✅ 완료!")
print(f"      - 최적 파라미터: {grid_search_multiclass.best_params_}")
print(f"      - 최고 F1-Macro 점수: {grid_search_multiclass.best_score_:.4f}")

# 최적 모델과 예측 결과 저장
best_multiclass_model = grid_search_multiclass.best_estimator_
multiclass_predictions = best_multiclass_model.predict(test_X_s1_selected)

print(f"      - 예측 완료: {len(multiclass_predictions)}개 샘플")
print(f"      - 예측된 클래스 분포: {dict(pd.Series(multiclass_predictions).value_counts().sort_index())}")

In [ ]:
# =============================================================================
# 7. 최종 제출 파일 생성
# =============================================================================

print(f"\n{'='*60}")
print(f"📋 최종 제출 파일 생성")
print(f"{'='*60}")

# sample 기반 제출 포맷 가져오기
submission_final = sample_submission[['subject_id', 'sleep_date', 'lifelog_date']].copy()

# lifelog_date 기준으로 string → date 형식 통일
submission_final['lifelog_date'] = pd.to_datetime(submission_final['lifelog_date']).dt.date

print(f"📝 제출 데이터 준비:")
print(f"   - 제출 샘플 수: {len(submission_final)}")

# S1 다중분류 예측 결과 사용
assert len(submission_final) == len(multiclass_predictions), f"길이 불일치: submission={len(submission_final)}, S1 predictions={len(multiclass_predictions)}"

# 다중 분류 예측 붙이기
submission_final['S1'] = multiclass_predictions

# 이진 분류 결과 붙이기
for col in targets_binary:
    individual_predictions = binary_preds_optimized[col]
    
    # shape 체크
    assert len(submission_final) == len(individual_predictions), f"길이 불일치: submission={len(submission_final)}, {col} predictions={len(individual_predictions)}"
    
    submission_final[col] = individual_predictions.astype(int)

# 📊 예측 결과 요약 출력
print(f"\n📊 최종 예측 결과 요약:")
print("=" * 40)

for col in ['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']:
    value_counts = submission_final[col].value_counts().sort_index()
    total = len(submission_final)
    print(f"🔹 {col}:")
    for value, count in value_counts.items():
        percentage = (count / total) * 100
        print(f"   클래스 {value}: {count:,}개 ({percentage:.1f}%)")

# 최종 제출 형식 정렬
submission_final = submission_final[['subject_id', 'sleep_date', 'lifelog_date', 'Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']]

# 저장
submission_final.to_csv(submission_folder + submission_file, index=False)

print(f"\n✅ 제출 파일이 저장되었습니다: {submission_folder + submission_file}")


In [ ]:
# =============================================================================
# 8. 모델 성능 요약
# =============================================================================

print(f"\n{'='*60}")
print(f"🎉 고급 전처리 + LightGBM 모델링 완료!")
print(f"{'='*60}")

print(f"\n📈 모델 성능 요약:")
print("-" * 50)

print(f"🔹 이진분류 모델들:")
for model_name in targets_binary:
    if model_name in binary_models_optimized:
        print(f"   - {model_name}: ✅ 학습 완료")
    else:
        print(f"   - {model_name}: ❌ 학습 실패")

print(f"\n🔹 다중분류 모델:")
print(f"   - S1 (다중분류): ✅ 학습 완료")
print(f"   - 최고 F1-Macro 점수: {grid_search_multiclass.best_score_:.4f}")

print(f"\n📊 사용된 특성 정보:")
print(f"   - 전체 특성 수: {len(feature_cols)}")
print(f"   - 고급 시계열 특성: {len([col for col in feature_cols if 'rolling' in col or 'lag' in col])}")
print(f"   - 개인화 특성: {len([col for col in feature_cols if 'personal' in col])}")
print(f"   - 생체신호 특성: {len([col for col in feature_cols if any(bio in col for bio in ['hr_', 'rmssd', 'pnn50'])])}")
print(f"   - 행동패턴 특성: {len([col for col in feature_cols if any(pattern in col for pattern in ['session', 'pattern', 'transition'])])}")

print(f"\n💾 저장된 결과:")
print(f"   - 제출 파일: {submission_file}")
print(f"   - 위치: {submission_folder}")

# 모델 객체들도 저장되어 추후 분석 가능
print(f"\n🔧 사용 가능한 모델 객체들:")
print(f"   - binary_models_optimized: 최적화된 이진분류 모델들")
print(f"   - best_multiclass_model: 최적화된 S1 다중분류 모델")
print(f"   - basic_importance_dict: 각 모델별 Feature Importance")

print(f"\n🎯 다음 단계 제안:")
print(f"   1. 교차검증으로 모델 성능 검증")
print(f"   2. 앙상블 모델 적용 (Voting, Stacking)")
print(f"   3. 하이퍼파라미터 추가 튜닝")
print(f"   4. 특성 선택 최적화")

print(f"{'='*60}")